# ⚽ Football League Predictor - Training Notebook

Google Colab GPU環境での深層学習モデル学習用ノートブック

## 🎯 目的
- 複数リーグのデータで統合学習
- オッズに依存しないPPG・xGベースの予測
- 継続学習・ファインチューニング対応
- PyTorchモデルの保存・読み込み

## 🚀 セットアップ

In [ ]:
# Google Colab環境のセットアップ
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install pytorch-lightning
!pip install pandas numpy matplotlib seaborn scikit-learn
!pip install pyyaml hydra-core

# GPU確認
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch Version: {torch.__version__}")

In [ ]:
# Google Driveマウント（データとモデル保存用）
from google.colab import drive
drive.mount('/content/drive')

# プロジェクトディレクトリ設定（統一）
import os
PROJECT_DIR = '/content/drive/MyDrive/league-predictor'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.chdir(PROJECT_DIR)

print(f"Working directory: {os.getcwd()}")
print("📁 Google Driveマウント完了")

In [ ]:
# GitHubリポジトリクローン（初回のみ）
if not os.path.exists('league-predictor'):
    !git clone https://github.com/2245093t/league-predictor.git
    print("✅ GitHubリポジトリをクローンしました")
else:
    print("✅ GitHubリポジトリは既に存在します")
    
# league-predictorディレクトリに移動
os.chdir('league-predictor')
print(f"作業ディレクトリ: {os.getcwd()}")

# ディレクトリ構造確認
print("\n📁 プロジェクト構造:")
!ls -la

print("\n📁 srcディレクトリの内容:")
if os.path.exists('src'):
    !ls -la src/
    print("\n📁 src/training の内容:")
    if os.path.exists('src/training'):
        !ls -la src/training/
else:
    print("❌ srcディレクトリが見つかりません")

## 📊 データ確認

Google Drive上の`stats-csv`フォルダに直接保存されたリーグデータを確認します。

In [ ]:
# データ確認（Google Drive統合管理）
import pandas as pd
import glob
import os

# CSVファイルディレクトリの設定
CSV_DIR = "/content/drive/MyDrive/league-predictor/stats-csv"
os.makedirs(CSV_DIR, exist_ok=True)

# 既存のCSVファイル確認
existing_files = glob.glob(os.path.join(CSV_DIR, "*.csv"))
print("=" * 60)
print("📊 GOOGLE DRIVE データ確認")
print("=" * 60)
print(f"📁 データディレクトリ: {CSV_DIR}")
print(f"📄 検出されたCSVファイル: {len(existing_files)}個")

if existing_files:
    print("\n🏆 リーグデータファイル一覧:")
    total_matches = 0
    
    for i, file in enumerate(existing_files, 1):
        filename = os.path.basename(file)
        df = pd.read_csv(file)
        file_size = os.path.getsize(file) / (1024 * 1024)  # MB
        total_matches += len(df)
        
        print(f"  {i}. {filename}")
        print(f"     試合数: {len(df):,}")
        print(f"     サイズ: {file_size:.1f}MB")
        print(f"     カラム数: {len(df.columns)}")
    
    print(f"\n📊 総計:")
    print(f"  総試合数: {total_matches:,}")
    print(f"  総ファイル数: {len(existing_files)}")
    
    print("\n✅ 全リーグのデータが統合準備完了！")
    print("🌍 各リーグのデータが対等に扱われます")
else:
    print("\n❌ CSVファイルが見つかりません")
    print("📝 新しいリーグデータを追加するには:")
    print("   1. Google Driveで以下のフォルダにアクセス")
    print(f"   2. {CSV_DIR}")
    print("   3. リーグのCSVファイルを直接アップロード")
    print("   4. このセルを再実行してデータを確認")

print(f"\n💡 新しいリーグデータを追加したい場合:")
print(f"   Google Driveの {CSV_DIR} フォルダに")
print("   CSVファイルを直接アップロードしてください")

In [ ]:
# 統合データ概要確認
import glob
import os
import sys
import pandas as pd

# srcフォルダのパスを追加
sys.path.append('src')
sys.path.append('/content/drive/MyDrive/league-predictor/src')

from training.data_loader import UnifiedDataLoader

def analyze_unified_data():
    """統合されたデータの概要を分析"""
    
    config = {'batch_size': 64, 'feature_dim': 11}
    loader = UnifiedDataLoader(config)
    
    # 統計情報取得
    stats = loader.get_league_statistics(CSV_DIR)
    
    print("=" * 50)
    print("🌍 UNIFIED LEAGUE DATA ANALYSIS")
    print("=" * 50)
    
    print(f"📊 全体統計:")
    print(f"  総試合数: {stats['total_matches']:,}")
    print(f"  総チーム数: {stats['total_teams']:,}")
    
    print(f"\n🏆 リーグ別分布:")
    for league, count in stats['league_distribution'].items():
        percentage = (count / stats['total_matches']) * 100
        print(f"  {league}: {count:,}試合 ({percentage:.1f}%)")
    
    print(f"\n⚽ リーグ別チーム数:")
    for league, count in stats['teams_per_league'].items():
        print(f"  {league}: {count}チーム")
    
    # データ品質チェック
    csv_files = glob.glob(os.path.join(CSV_DIR, "*.csv"))
    print(f"\n📁 CSVファイル詳細:")
    
    total_size = 0
    for csv_file in csv_files:
        filename = os.path.basename(csv_file)
        file_size = os.path.getsize(csv_file) / (1024 * 1024)  # MB
        total_size += file_size
        
        df = pd.read_csv(csv_file)
        print(f"  {filename}")
        print(f"    サイズ: {file_size:.1f}MB")
        print(f"    試合数: {len(df):,}")
        print(f"    カラム数: {len(df.columns)}")
    
    print(f"\n💾 総データサイズ: {total_size:.1f}MB")
    print("\n✅ 全リーグのデータが統合準備完了！")

analyze_unified_data()

## ⚙️ 学習設定

In [ ]:
# 統合学習設定
TRAINING_CONFIG = {
    # モデルアーキテクチャ
    'num_teams': 500,        # 多リーグ統合のため大幅増加
    'embedding_dim': 64,     # チーム埋め込み次元増加
    'hidden_dim': 512,       # 隠れ層次元増加
    'dropout': 0.3,          # ドロップアウト率
    
    # 学習パラメータ
    'learning_rate': 0.001,
    'weight_decay': 0.01,
    'batch_size': 128,       # 大量データのためバッチサイズ増加
    'epochs': 150,           # エポック数調整
    
    # 損失関数重み
    'goal_loss_weight': 1.0,
    'result_loss_weight': 2.0,
    
    # デバイス
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Google Drive設定
DRIVE_CONFIG = {
    'csv_dir': CSV_DIR,
    'model_save_dir': '/content/drive/MyDrive/league-predictor/models',
    'encoder_save_path': '/content/drive/MyDrive/league-predictor/models/team_encoder.json'
}

# モデル保存ディレクトリ作成
os.makedirs(DRIVE_CONFIG['model_save_dir'], exist_ok=True)

print("=" * 50)
print("📋 統合学習設定")
print("=" * 50)
print(f"📍 CSVデータディレクトリ: {DRIVE_CONFIG['csv_dir']}")
print(f"💾 モデル保存ディレクトリ: {DRIVE_CONFIG['model_save_dir']}")
print(f"🔧 使用デバイス: {TRAINING_CONFIG['device']}")
print(f"🧠 最大チーム数: {TRAINING_CONFIG['num_teams']}")
print(f"📦 バッチサイズ: {TRAINING_CONFIG['batch_size']}")
print(f"🔄 エポック数: {TRAINING_CONFIG['epochs']}")
print()
print("✅ 全リーグが対等に扱われる統合学習の準備完了！")

## 🧠 モデル学習

In [ ]:
# 統合学習モジュールのインポート
import sys
import os
import numpy as np

# srcフォルダのパスを追加（複数の可能性に対応）
sys.path.append('src')
sys.path.append('/content/drive/MyDrive/league-predictor/src')
sys.path.append('/content/drive/MyDrive/league-predictor/league-predictor/src')

# 現在の作業ディレクトリを確認
print(f"Current working directory: {os.getcwd()}")
print(f"Python path: {sys.path[-3:]}")

try:
    # 正しいクラス名でインポート
    from training.data_loader import UnifiedDataLoader
    from training.train_model import FootballTrainer
    from training.model_architecture import FootballMatchPredictor
    
    print("✅ 統合データローダーのインポート完了")
    print("🌍 全リーグのデータが対等に扱われるシステムです")
    
except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    print("📁 利用可能なファイルを確認します:")
    
    # srcフォルダの内容確認
    if os.path.exists('src'):
        print("src/ フォルダの内容:")
        for root, dirs, files in os.walk('src'):
            level = root.replace('src', '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files:
                if file.endswith('.py'):
                    print(f"{subindent}{file}")
    else:
        print("src/ フォルダが見つかりません")
    
    # 具体的なファイル内容確認
    data_loader_path = 'src/training/data_loader.py'
    if os.path.exists(data_loader_path):
        print(f"\n📄 {data_loader_path} の内容を確認:")
        with open(data_loader_path, 'r') as f:
            lines = f.readlines()[:20]  # 最初の20行を表示
            for i, line in enumerate(lines, 1):
                if 'class' in line or 'def' in line:
                    print(f"  {i}: {line.strip()}")
    
    # 直接インポートを試行
    print("\n🔧 直接インポートを試行:")
    try:
        import importlib.util
        spec = importlib.util.spec_from_file_location("data_loader", data_loader_path)
        data_loader_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(data_loader_module)
        
        # 利用可能なクラスを確認
        classes = [name for name in dir(data_loader_module) if name[0].isupper()]
        print(f"  利用可能なクラス: {classes}")
        
        # UnifiedDataLoaderクラスを取得
        if hasattr(data_loader_module, 'UnifiedDataLoader'):
            UnifiedDataLoader = getattr(data_loader_module, 'UnifiedDataLoader')
            print("  ✅ UnifiedDataLoader を直接読み込み成功")
        else:
            print("  ❌ UnifiedDataLoader クラスが見つかりません")
            
    except Exception as direct_error:
        print(f"  ❌ 直接読み込みエラー: {direct_error}")

In [ ]:
# 事前学習済みモデルの確認
pretrained_path = 'models/saved/best_model.pth'

if os.path.exists(pretrained_path):
    print(f"事前学習済みモデルが見つかりました: {pretrained_path}")
    
    # モデル情報確認
    checkpoint = torch.load(pretrained_path, map_location='cpu')
    if 'metadata' in checkpoint:
        metadata = checkpoint['metadata']
        print(f"前回の学習エポック: {metadata.get('epoch', 'Unknown')}")
        print(f"検証損失: {metadata.get('val_loss', 'Unknown')}")
        print(f"精度: {metadata.get('accuracy', 'Unknown')}")
        
    use_pretrained = True
else:
    print("事前学習済みモデルが見つかりません。新規学習を開始します。")
    pretrained_path = None
    use_pretrained = False

In [ ]:
# 統合学習開始
print("=" * 60)
print("🌍 UNIFIED MULTI-LEAGUE TRAINING")
print("=" * 60)

# 必要なクラスがインポートされているか確認
required_classes = ['UnifiedDataLoader', 'FootballTrainer', 'FootballMatchPredictor']
missing_classes = []

for class_name in required_classes:
    if class_name not in globals():
        missing_classes.append(class_name)

if missing_classes:
    print(f"❌ 未インポートのクラス: {missing_classes}")
    print("🔧 緊急インポートを実行します...")
    
    # 直接インポートを試行
    try:
        import sys
        import importlib.util
        
        # data_loader.pyから直接インポート
        data_loader_path = 'src/training/data_loader.py'
        spec = importlib.util.spec_from_file_location("data_loader", data_loader_path)
        data_loader_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(data_loader_module)
        UnifiedDataLoader = getattr(data_loader_module, 'UnifiedDataLoader')
        
        # train_model.pyから直接インポート
        train_model_path = 'src/training/train_model.py'
        spec = importlib.util.spec_from_file_location("train_model", train_model_path)
        train_model_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(train_model_module)
        FootballTrainer = getattr(train_model_module, 'FootballTrainer')
        
        # model_architecture.pyから直接インポート
        model_arch_path = 'src/training/model_architecture.py'
        spec = importlib.util.spec_from_file_location("model_architecture", model_arch_path)
        model_arch_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(model_arch_module)
        FootballMatchPredictor = getattr(model_arch_module, 'FootballMatchPredictor')
        
        print("✅ 緊急インポート成功!")
        
    except Exception as import_error:
        print(f"❌ 緊急インポートエラー: {import_error}")
        print("第11セル（統合学習モジュールインポート）を先に実行してください")
        raise ImportError("必要なクラスがインポートされていません")

# 統合データローダー初期化
data_loader = UnifiedDataLoader(TRAINING_CONFIG)

# 事前学習済みエンコーダー読み込み（あれば）
encoder_path = DRIVE_CONFIG['encoder_save_path']
if os.path.exists(encoder_path):
    data_loader.load_team_encoder(encoder_path)
    print(f"✅ 既存のチームエンコーダーを読み込み: {len(data_loader.team_encoder)}チーム")

# 統合データセット作成
print(f"📊 Google Driveからデータ読み込み: {DRIVE_CONFIG['csv_dir']}")
train_dataloader = data_loader.load_from_drive(DRIVE_CONFIG['csv_dir'])

# チームエンコーダー保存
data_loader.save_team_encoder(DRIVE_CONFIG['encoder_save_path'])
print(f"💾 チームエンコーダー保存: {DRIVE_CONFIG['encoder_save_path']}")

# トレーナー初期化
trainer = FootballTrainer(TRAINING_CONFIG)

# 事前学習済みモデル読み込み（あれば）
pretrained_model_path = os.path.join(DRIVE_CONFIG['model_save_dir'], 'best_model.pth')
if os.path.exists(pretrained_model_path) and use_pretrained:
    trainer.initialize_model(pretrained_model_path)
    print(f"🔄 継続学習モード: {pretrained_model_path}")
else:
    trainer.initialize_model(None)
    print("🆕 新規学習モード")

# 学習実行
print("\n🚀 統合学習開始...")
trainer.train_with_unified_data(
    train_dataloader=train_dataloader,
    validation_split=0.2,
    save_path=DRIVE_CONFIG['model_save_dir'],
    checkpoint_interval=25
)

print("✅ 統合学習完了！")
print("🏆 全リーグのパターンを学習したモデルが完成しました")

## 📈 モデル評価

In [ ]:
# 学習履歴の可視化
import matplotlib.pyplot as plt

def plot_training_history(history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 損失
    axes[0, 0].plot(history['epoch'], history['train_loss'], label='Train Loss')
    axes[0, 0].plot(history['epoch'], history['val_loss'], label='Validation Loss')
    axes[0, 0].set_title('Total Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    
    # ゴール予測損失
    axes[0, 1].plot(history['epoch'], history['goal_loss'], label='Goal Loss', color='orange')
    axes[0, 1].set_title('Goal Prediction Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('MSE Loss')
    
    # 結果予測損失
    axes[1, 0].plot(history['epoch'], history['result_loss'], label='Result Loss', color='green')
    axes[1, 0].set_title('Result Prediction Loss')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('CrossEntropy Loss')
    
    # 精度
    axes[1, 1].plot(history['epoch'], history['accuracy'], label='Accuracy', color='red')
    axes[1, 1].set_title('Classification Accuracy')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Accuracy')
    
    plt.tight_layout()
    plt.show()

# 学習履歴プロット
if trainer.train_history['epoch']:
    plot_training_history(trainer.train_history)
else:
    print("学習履歴が見つかりません")

In [ ]:
# モデルテスト予測
def test_model_prediction():
    # Google Drive上の正しいパスを使用
    model_path = os.path.join(DRIVE_CONFIG['model_save_dir'], 'best_model.pth')
    encoder_path = DRIVE_CONFIG['encoder_save_path']
    
    # パス確認
    print(f"🔍 モデルパス: {model_path}")
    print(f"🔍 エンコーダーパス: {encoder_path}")
    
    if not os.path.exists(model_path):
        print(f"❌ モデルファイルが見つかりません: {model_path}")
        print("利用可能なモデルファイル:")
        model_dir = DRIVE_CONFIG['model_save_dir']
        if os.path.exists(model_dir):
            for file in os.listdir(model_dir):
                if file.endswith('.pth'):
                    print(f"  {file}")
        return
    
    if not os.path.exists(encoder_path):
        print(f"❌ エンコーダーファイルが見つかりません: {encoder_path}")
        return
    
    # 最新モデル読み込み
    model = FootballMatchPredictor.load_model(
        model_path, 
        device=TRAINING_CONFIG['device']
    )
    
    # チームエンコーダー読み込み
    import json
    with open(encoder_path, 'r', encoding='utf-8') as f:
        team_encoder = json.load(f)
    
    print(f"✅ モデル読み込み完了")
    print(f"✅ チームエンコーダー読み込み完了: {len(team_encoder)}チーム")
    
    # サンプル予測
    sample_matches = [
        {
            'home_team': 'Manchester City',
            'away_team': 'Liverpool',
            'features': np.array([2.8, 2.7, 2.9, 2.6, 2.2, 2.0, 1.0, 1.1, 0.5, 0.3, 0.1])
        },
        {
            'home_team': 'Arsenal', 
            'away_team': 'Chelsea',
            'features': np.array([2.1, 2.0, 2.2, 1.9, 1.8, 1.7, 1.2, 1.3, 0.6, 0.3, -0.1])
        }
    ]
    
    print("\n🔮 サンプル予測結果:")
    print("=" * 50)
    
    for match in sample_matches:
        home_team = match['home_team']
        away_team = match['away_team']
        
        if home_team in team_encoder and away_team in team_encoder:
            home_id = team_encoder[home_team]
            away_id = team_encoder[away_team]
            
            prediction = model.predict_match(home_id, away_id, match['features'])
            
            print(f"\n{home_team} vs {away_team}")
            print(f"予測スコア: {prediction['home_goals']:.1f} - {prediction['away_goals']:.1f}")
            print(f"勝利確率: ホーム {prediction['home_win_prob']:.1%}, "
                  f"ドロー {prediction['draw_prob']:.1%}, "
                  f"アウェイ {prediction['away_win_prob']:.1%}")
            print(f"予測結果: {prediction['predicted_result']}")
        else:
            print(f"❌ チーム {home_team} または {away_team} がエンコーダーに見つかりません")
            print(f"  利用可能なチーム例: {list(team_encoder.keys())[:10]}...")

test_model_prediction()

## 💾 モデル保存とダウンロード

In [ ]:
# 最終モデルを日付付きで保存
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_name = f'football_predictor_{timestamp}.pth'
final_model_path = os.path.join(DRIVE_CONFIG['model_save_dir'], final_model_name)

# 最新のベストモデルをコピー
best_model_path = os.path.join(DRIVE_CONFIG['model_save_dir'], 'best_model.pth')

if os.path.exists(best_model_path):
    !cp "{best_model_path}" "{final_model_path}"
    print(f"✅ 最終モデル保存: {final_model_path}")
else:
    print(f"❌ ベストモデルが見つかりません: {best_model_path}")

# モデルファイル一覧
print(f"\n📁 保存済みモデル ({DRIVE_CONFIG['model_save_dir']}):")
if os.path.exists(DRIVE_CONFIG['model_save_dir']):
    !ls -lh "{DRIVE_CONFIG['model_save_dir']}"/*.pth
else:
    print("モデル保存ディレクトリが見つかりません")

In [ ]:
# モデルとエンコーダーをダウンロード
from google.colab import files

# ダウンロード対象ファイル（Google Drive上のフルパスを使用）
download_files = [
    os.path.join(DRIVE_CONFIG['model_save_dir'], 'best_model.pth'),
    DRIVE_CONFIG['encoder_save_path'],
    final_model_path  # 前のセルで定義された変数
]

print("📦 ダウンロード開始...")
print(f"📁 対象ディレクトリ: {DRIVE_CONFIG['model_save_dir']}")

for file_path in download_files:
    if os.path.exists(file_path):
        try:
            files.download(file_path)
            filename = os.path.basename(file_path)
            print(f"✅ {filename} ダウンロード完了")
        except Exception as e:
            print(f"❌ {os.path.basename(file_path)} ダウンロードエラー: {e}")
    else:
        print(f"❌ ファイルが見つかりません: {os.path.basename(file_path)}")

print("\n? ダウンロード完了したファイル:")
print("1. best_model.pth - 最高性能モデル")
print("2. team_encoder.json - チーム名とIDのマッピング")
print("3. football_predictor_[timestamp].pth - タイムスタンプ付きバックアップ")
print("\n💡 これらのファイルをローカルの models/saved/ ディレクトリに配置してください")

## 🎯 次のステップ

1. **ローカル環境での予測**: ダウンロードしたモデルをローカル環境で使用
2. **新データでファインチューニング**: 新しいリーグデータが追加されたら継続学習
3. **予測精度の評価**: 実際の試合結果と比較して精度を検証
4. **自動化**: 定期的な学習とモデル更新の自動化

### 継続学習の実行方法
```python
# 新しいデータを追加後、このノートブックを再実行
# 事前学習済みモデルが自動的に読み込まれ、ファインチューニングが実行されます
```

### ローカルでの使用方法
```python
from src.prediction.predict_matches import predict_weekly_matches

# 週次予測の実行
predictions = predict_weekly_matches('data/fixtures/current_season.csv')
```